In [1]:
import os.path

from lab3.benchmark.dataset_generator import generate, GeneratorConfig, Functions
from lab3.benchmark.transport import get_stub
from lab3.benchmark.visualization import plot_model_over_dataset
from transport.generated.api_pb2 import TrainRequest, TrainerConfig, ModelConfig
from lab3.benchmark.benchmark_result import train_and_test, BenchmarkResult


### Зависимость от скорости обучения 

In [2]:
name = "learning_rate_1dim.csv"
_, train, test = generate(name,
                          GeneratorConfig(dim=1, rows=200, noize=2, functions=Functions.LINEAR, split=(0.7, 0.3)))


def experiment(idx):
    rate = 0.001 * (idx ** 2)
    f = lambda: train_and_test("Test learning rate", test, TrainRequest(
        path=train,
        trainerConfig=TrainerConfig(type="GreedyTrainer", params=[0.01, 100000, rate]),
        modelConfig=ModelConfig(type="LinearModel", regularizator="EmptyRegularizator", loss="MSELoss"),
    ))
    f.__name__ = f"rate-{rate}"
    return f


fs = []
for i in range(1, 10):
    fs.append(experiment(i))

r = BenchmarkResult.compare(fs)
r.top(*r.all(), total=False)

--running rate-0.001


TypeError: _UnaryUnaryMultiCallable.__call__() missing 1 required positional argument: 'request'